<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

Wide World Importers is interested in ingesting the data from web logs from an HDFS source where they have been streamed. They want to be able to analyze the traffic to see if there is a pattern in time, products or locations. 

The web logs, however, are refreshed periodically. WWI would like to keep the logs in local storage to do deeper analysis. 

In this Jupyter Notebook you'll create a location to store the log files as a SQL Server Table in the SQL Data Pool, and then fill it by creating an External Table that reads HDFS.

In [3]:
USE WideWorldImporters;
GO

CREATE EXTERNAL DATA SOURCE SqlDataPool
WITH (LOCATION = 'sqldatapool://controller-svc/default');

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.166

In [4]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
    );
GO

Commands completed successfully.

Total execution time: 00:00:08.849

In [5]:
/* Create an External Table that can read from the Storage Pool File Location */
IF NOT EXISTS(SELECT * FROM sys.external_tables WHERE name = 'web_clickstreams_hdfs')
BEGIN
    CREATE EXTERNAL TABLE [web_clickstreams_hdfs]
    ("wcs_click_date_sk" BIGINT 
    , "wcs_click_time_sk" BIGINT 
    , "wcs_sales_sk" BIGINT 
    , "wcs_item_sk" BIGINT
    , "wcs_web_page_sk" BIGINT 
    , "wcs_user_sk" BIGINT)
    WITH
    (
        DATA_SOURCE = SqlStoragePool,
        LOCATION = '/web_logs',
        FILE_FORMAT = csv_file
    );
END

Commands completed successfully.

Total execution time: 00:00:00.223

In [6]:
BEGIN
   INSERT INTO web_clickstream_clicks_data_pool
   SELECT wcs_click_date_sk
    , wcs_click_time_sk 
    , wcs_sales_sk 
    , wcs_item_sk 
    , wcs_web_page_sk  
    , wcs_user_sk 
     FROM web_clickstreams_hdfs
END

(6770549 rows affected)

Total execution time: 00:00:42.670

In [7]:
SELECT count(*) FROM [dbo].[web_clickstream_clicks_data_pool]
SELECT TOP 10 * FROM [dbo].[web_clickstream_clicks_data_pool]

(1 row affected)

(10 rows affected)

Total execution time: 00:00:00.843

(No column name)
6770549


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
37775,35460,NULL,7394,53,NULL
37775,12155,NULL,13157,53,NULL
37775,4880,NULL,13098,53,NULL
37775,36272,NULL,6851,53,NULL
37775,24922,NULL,5198,53,NULL
37776,74100,NULL,16015,53,NULL
37776,26833,NULL,12921,53,NULL
37776,72943,NULL,5015,53,NULL
37776,9387,NULL,12274,53,NULL
37776,32557,NULL,11344,53,NULL


## Next Steps: Continue on to Working with Spark and ETL

Now you're ready to open the next Python Notebook - `bdc_tutorial_04.ipynb` - to learn how to create and work with Spark and Extracting, Transforming and Loading data.